In [20]:
import math

class SimpleRandom:
    def __init__(self, seed=42):
        self.modulus = 2**32
        self.a = 1664525
        self.c = 1013904223
        self.state = seed

    def rand_float(self):
        self.state = (self.a * self.state + self.c) % self.modulus
        return self.state / self.modulus

rng = SimpleRandom()

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

class Node:
    def __init__(self, input_size, weights=None, bias=None):
        if weights:
            self.weights = weights
        else:
            self.weights = [rng.rand_float() for _ in range(input_size)]
        self.bias = bias if bias is not None else rng.rand_float()

    def forward(self, inputs):
        total = sum(w * i for w, i in zip(self.weights, inputs)) + self.bias
        return sigmoid(total)

class Layer:
    def __init__(self, input_size, output_size, weights=None, biases=None):
        self.nodes = []
        for i in range(output_size):
            w = weights[i] if weights else None
            b = biases[i] if biases else None
            self.nodes.append(Node(input_size, w, b))

    def forward(self, inputs):
        return [node.forward(inputs) for node in self.nodes]

class NeuralNetwork:
    def __init__(self, layer_sizes, init_type="random", weight_file=None):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            if init_type == "file" and weight_file:
                weights, biases = self._load_weights(weight_file, i)
                layer = Layer(layer_sizes[i], layer_sizes[i+1], weights, biases)
            else:
                layer = Layer(layer_sizes[i], layer_sizes[i+1])
            self.layers.append(layer)

    def _load_weights(self, file, layer_index):
        with open(file, 'r') as f:
            lines = f.readlines()
        start = 0
        for i in range(layer_index):
            n_nodes = int(lines[start])
            start += 1 + n_nodes
        n_nodes = int(lines[start])
        start += 1
        weights = [list(map(float, lines[start+i].split()))[:-1] for i in range(n_nodes)]
        biases = [float(lines[start+i].split()[-1]) for i in range(n_nodes)]
        return weights, biases

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

def load_config(file):
    with open(file) as f:
        lines = list(map(int, f.readlines()))
    return lines



In [21]:
if __name__ == "__main__":
    arch = load_config("config.txt")
    nn = NeuralNetwork(arch, init_type="random")

    inputs = [(0,0), (0,1), (1,0), (1,1)]
    print("XOR Results:")
    for x in inputs:
        output = nn.forward(list(x))
        print(f"Input: {x} -> Output: {output[0]:.4f}")

XOR Results:
Input: (0, 0) -> Output: 0.6749
Input: (0, 1) -> Output: 0.6754
Input: (1, 0) -> Output: 0.6773
Input: (1, 1) -> Output: 0.6776
